# Harder task - possibly homework
* your task is to convert above text into a pandas dataframe
* using just Selenium to make a list of dictionaries
* where each dictionary is a row in the dataframe
* and each key is a column name
* and each value is a value in the column

* save pandas to csv file- you already know this

In [83]:
# let's import the selenium library
import selenium
# let's import the webdriver
from selenium import webdriver
# let's import the keys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# let's import the time
import time
# let's import the pandas for later use
import pandas as pd
from bs4 import BeautifulSoup

import requests

In [59]:
browser=webdriver.Chrome()

In [60]:
browser.get('http://www.ss.com/en')
assert 'SS' in browser.title 

In [61]:
offices = browser.find_element(By.ID, "mtd_5457")
type(offices)

selenium.webdriver.remote.webelement.WebElement

In [62]:
offices.click()

In [63]:
jelgava = browser.find_element(By.XPATH, '//a[@title="Jelgava and reg., Advertisements"]')
type(jelgava)

selenium.webdriver.remote.webelement.WebElement

In [64]:
jelgava.click()

In [65]:
browser.current_url

'https://www.ss.com/en/real-estate/offices/jelgava-and-reg/'

In [66]:
tables = browser.find_elements(By.TAG_NAME, 'table')
len(tables)
type(tables)


list

In [67]:
tables[1].text

'Offices / Jelgava and reg.\nAll advertisements\nFor 2 days - 4\nFor 5 days - 7\n\n\nPrice:\n-  Area:\n-  Floor:\n- \nMode:\n    List\n    Album\n    Map\n     Deal type:\nAll\nSell\nHand over\nCity, civil parish:\nAll\nJelgava\nOzolnieku pag.\nStreet:\n Advertisements\ndate\nCity/civil parish m2 Floor Price, m2 Price\nTelpas Pērnavas ielā 10. Помещение под офис или учебный класс Telpa\nJelgava 50 1/5 4 € 200 €/mon.\nIzīrē kabinetu 30m2, Jelgavā. Telpā ir ūdens padeve ar kanalizācij\nJelgava 30 2/3 5 € 150 €/mon.\nIznomā biroja telpas, sastāv no pieņemamās telpas/gaiteņa un 3 kabin\nJelgava 53 4/4 5.02 € 266 €/mon.\nIzīrē telpas Jūgenstila namā -1 stāvā, ieeja atsevišķa no Zemgales p\nJelgava 80 1/4 6.24 € 499 €/mon.\nPārdod vai izīrē telpas Satiksmes ielā 33, Jelgava. Blakus liels aut\nJelgava 25 1/9 740 € 18,500 €\nIzīrē mēbelētu biroja telpu ēkas 4.stāvā, telpa aprīkota ar kondicio\nJelgava 21 4/6 4 € 84 €/mon.\nIznoma komerctelpas Raf rajonā ar kopējo platību 74.60m2. Telpas pi\nJe

In [68]:
second_header = browser.find_element(By.XPATH, '//*[@id="table"]/tbody/tr[2]/th').text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="table"]/tbody/tr[2]/th"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x0000000102384598 chromedriver + 4404632
1   chromedriver                        0x000000010230bfa3 chromedriver + 3911587
2   chromedriver                        0x0000000101fb8e6a chromedriver + 425578
3   chromedriver                        0x0000000101ff0c91 chromedriver + 654481
4   chromedriver                        0x0000000101ff0e91 chromedriver + 654993
5   chromedriver                        0x0000000102028274 chromedriver + 881268
6   chromedriver                        0x000000010200edcd chromedriver + 777677
7   chromedriver                        0x0000000102025f79 chromedriver + 872313
8   chromedriver                        0x000000010200ebd3 chromedriver + 777171
9   chromedriver                        0x0000000101fe3ced chromedriver + 601

In [70]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [71]:
def getColList(soup):
    column_list = ["description","url"] # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"}) #this will work as long as html has a table row with id head_line
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]]  # this will get all column names starting with 2nd in HTML
    column_list += headcolumns
    return column_list

In [77]:
column_names = getColList(soup)
column_names

['description',
 'url',
 'City/civil parish',
 'm2',
 'Floor',
 'Price, m2',
 'Price']

In [78]:
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [79]:
def getRow(row, colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [80]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist]

In [81]:
def getDFfromURL(url):
    # print("getting data from", url)
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None # maybe return empty dataframe here
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows)

In [84]:
idf = getDFfromURL(browser.current_url)
idf.head()

,description,url,City/civil parish,m2,Floor,"Price, m2",Price
0,Telpas Pērnavas ielā 10. Помещение под офис ил...,https://ss.com/msg/en/real-estate/offices/jelg...,Jelgava,50,1/5,4 €,200 €/mon.
1,"Izīrē kabinetu 30m2, Jelgavā. \r\nTelpā ir ūde...",https://ss.com/msg/en/real-estate/offices/jelg...,Jelgava,30,2/3,5 €,150 €/mon.
2,"Iznomā biroja telpas, sastāv no pieņemamās tel...",https://ss.com/msg/en/real-estate/offices/jelg...,Jelgava,53,4/4,5.02 €,266 €/mon.
3,"Izīrē telpas Jūgenstila namā -1 stāvā, ieeja a...",https://ss.com/msg/en/real-estate/offices/jelg...,Jelgava,80,1/4,6.24 €,499 €/mon.
4,"Pārdod vai izīrē telpas Satiksmes ielā 33, Jel...",https://ss.com/msg/en/real-estate/offices/jelg...,Jelgava,25,1/9,740 €,"18,500 €"


In [ ]:
idf.to_csv("jalgava_office_list.csv")